<a href="https://colab.research.google.com/github/mauryadivya/Data-Preprocessing-EDA-and-Model-Building/blob/main/Cointab_Data_Analysis_F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [233]:
# import libraries

import pandas as pd
import numpy as np


In [234]:
# read files

df_order=pd.read_excel("/content/Company X - Order Report.xlsx")
df_pincode=pd.read_excel("/content/Company X - Pincode Zones.xlsx")
df_sku=pd.read_excel("/content/Company X - SKU Master.xlsx")
df_invoice=pd.read_excel("/content/Courier Company - Invoice.xlsx")
df_rates=pd.read_excel("/content/Courier Company - Rates.xlsx")

In [235]:
 # merge pincode data with courier data by column Customer Pincode

df_invoice_pincode_zone= df_invoice.merge(df_pincode, how = "right", on = "Customer Pincode" )

In [236]:
# merging two dataframes

df_order_weight = df_order.merge(df_sku, how = "right", on = "SKU" )

In [237]:
# calculated total weights for each order ID

df_order_weight["sku_total_weight"] = df_order_weight["Order Qty"]*df_order_weight["Weight (g)"]

In [238]:
# total weight for each order ID

df_total_weight = df_order_weight[["ExternOrderNo", "sku_total_weight"]].groupby("ExternOrderNo").sum().reset_index()

In [239]:
# converted units

df_total_weight["sku_total_weight"] = df_total_weight["sku_total_weight"]/1000
df_total_weight

,ExternOrderNo,sku_total_weight
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [240]:
# used ceil function to form a weight slab column for X company data

df_total_weight["weight_slab"] = np.ceil(df_total_weight["sku_total_weight"]*2)*0.5

In [241]:
df_total_weight

,ExternOrderNo,sku_total_weight,weight_slab
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


In [242]:
# rename externorderno to order ID

df_total_weight.rename(columns = {'ExternOrderNo':'Order ID'}, inplace = True)

In [243]:
df_total_weight

,Order ID,sku_total_weight,weight_slab
0,2001806210,0.220,0.5
1,2001806226,0.480,0.5
2,2001806229,0.500,0.5
3,2001806232,1.302,1.5
4,2001806233,0.245,0.5
...,...,...,...
119,2001821995,0.477,0.5
120,2001822466,1.376,1.5
121,2001823564,0.672,1.0
122,2001825261,1.557,2.0


In [244]:
 # merge pincode data with courier data by column Customer Pincode

df_invoice_pincode_zone_slab_wt= df_invoice_pincode_zone.merge(df_total_weight, how = "right", on = "Order ID" )

In [245]:
# rename columns

df_invoice_pincode_zone_slab_wt.rename(columns = {'sku_total_weight':'Total weight as per X (kg)'}, inplace = True)
df_invoice_pincode_zone_slab_wt.rename(columns = {'weight_slab':'Weight slab as per X (kg)'}, inplace = True)
df_invoice_pincode_zone_slab_wt.rename(columns = {'Charged Weight':'Total weight as per Courier Company'}, inplace = True)
df_invoice_pincode_zone_slab_wt.rename(columns = {'Zone_y':'Delivery zone as per X'}, inplace = True)
df_invoice_pincode_zone_slab_wt.rename(columns = {'Zone_x':'Delivery zone as per courier company'}, inplace = True)
df_invoice_pincode_zone_slab_wt.rename(columns = {'Billing Amount (Rs.)':'Charges billed by courier company(Rs)'}, inplace = True)



In [246]:
# weight slab charged by courier company

df_invoice_pincode_zone_slab_wt["Weight slab charged by courier Company(kg)"] = np.ceil(df_invoice_pincode_zone_slab_wt["Total weight as per Courier Company"]*2)*0.5

In [247]:
# dropped columns that are not needed

df_invoice_pincode_zone_slab_wt.drop(['Warehouse Pincode_x','Warehouse Pincode_y','Customer Pincode'], axis=1, inplace=True)

In [248]:
# remove duplicates

df_invoice_pincode_zone_slab_wt=df_invoice_pincode_zone_slab_wt.drop_duplicates()

In [249]:
# calculated values of forward charges and RTO charges and then used slab (Weight slab as per X (kg)) column data to calculate Rate (Expected Charge as per X(Rs)).


# forward charges
b1=61.3
d1=90.2
e1=112.1

# forward and RTO charges
b2=110.1
d2=176.3
e2=218.3

df_invoice_pincode_zone_slab_wt["Rate"] = " "

for i in range(124):
  zonee = df_invoice_pincode_zone_slab_wt["Delivery zone as per X"].iloc[i]

  shipee = df_invoice_pincode_zone_slab_wt["Type of Shipment"].iloc[i]

  slab = df_invoice_pincode_zone_slab_wt["Weight slab as per X (kg)"].iloc[i]


  if zonee == "b" and shipee == "Forward charges":
      new1 = slab * b1
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new1
      # print(new1)

  elif zonee == "d" and shipee == "Forward charges":
      new2 = slab * d1
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new2
      # print(new2)

  elif zonee == "e" and shipee == "Forward charges":
      new7 = slab * e1
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new7
      # print(new7)

  elif zonee == "b" and shipee == "Forward and RTO charges":
      new3 = slab * b2
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new3
      # print(new3)

  elif zonee == "d" and shipee == "Forward and RTO charges":
      new4 = slab * d2
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new4
      # print(new4)

  elif zonee == "e" and shipee == "Forward and RTO charges":
      new5 = slab * e2
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Rate"] = new5
      # print(new5)



In [250]:
# rename column

df_invoice_pincode_zone_slab_wt.rename(columns = {'Rate':'Expected Charge as per X(Rs)'}, inplace = True)

In [251]:
# Calculated Difference Between Expected Charges and Billed Charges(Rs.)

df_invoice_pincode_zone_slab_wt['Difference Between Expected Charges and Billed Charges(Rs.)'] = df_invoice_pincode_zone_slab_wt['Charges billed by courier company(Rs)'] - df_invoice_pincode_zone_slab_wt['Expected Charge as per X(Rs)']

In [252]:
# drop column Type of Shipment

df_invoice_pincode_zone_slab_wt.drop(['Type of Shipment'], axis=1)

,AWB Code,Order ID,Total weight as per Courier Company,Delivery zone as per courier company,Charges billed by courier company(Rs),Delivery zone as per X,Total weight as per X (kg),Weight slab as per X (kg),Weight slab charged by courier Company(kg),Expected Charge as per X(Rs),Difference Between Expected Charges and Billed Charges(Rs.)
0,1091117221940,2001806210,2.92,b,174.5,b,0.220,0.5,3.0,30.65,143.85
1,1091117222065,2001806226,0.68,d,90.2,d,0.480,0.5,1.0,45.1,45.1
2,1091117222080,2001806229,0.71,d,90.2,d,0.500,0.5,1.0,45.1,45.1
3,1091117222124,2001806232,1.30,d,135.0,d,1.302,1.5,1.5,135.3,-0.3
4,1091117222135,2001806233,0.78,b,61.3,b,0.245,0.5,1.0,30.65,30.65
...,...,...,...,...,...,...,...,...,...,...,...
169,1091121183730,2001821995,0.50,d,45.4,b,0.477,0.5,0.5,30.65,14.75
170,1091121305541,2001822466,1.10,d,135.0,b,1.376,1.5,1.5,91.95,43.05
171,1091121666133,2001823564,0.70,d,172.8,d,0.672,1.0,1.0,176.3,-3.5
172,1091121981575,2001825261,1.60,d,345.0,d,1.557,2.0,2.0,352.6,-7.6


In [253]:
# reset indexes

df_invoice_pincode_zone_slab_wt.reset_index(inplace=True)

In [254]:
# created new column Charge Status

df_invoice_pincode_zone_slab_wt["Charge Status"] = " "

In [255]:
df_invoice_pincode_zone_slab_wt

,index,AWB Code,Order ID,Total weight as per Courier Company,Delivery zone as per courier company,Type of Shipment,Charges billed by courier company(Rs),Delivery zone as per X,Total weight as per X (kg),Weight slab as per X (kg),Weight slab charged by courier Company(kg),Expected Charge as per X(Rs),Difference Between Expected Charges and Billed Charges(Rs.),Charge Status
0,0,1091117221940,2001806210,2.92,b,Forward charges,174.5,b,0.220,0.5,3.0,30.65,143.85,
1,1,1091117222065,2001806226,0.68,d,Forward charges,90.2,d,0.480,0.5,1.0,45.1,45.1,
2,2,1091117222080,2001806229,0.71,d,Forward charges,90.2,d,0.500,0.5,1.0,45.1,45.1,
3,3,1091117222124,2001806232,1.30,d,Forward charges,135.0,d,1.302,1.5,1.5,135.3,-0.3,
4,4,1091117222135,2001806233,0.78,b,Forward charges,61.3,b,0.245,0.5,1.0,30.65,30.65,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,169,1091121183730,2001821995,0.50,d,Forward charges,45.4,b,0.477,0.5,0.5,30.65,14.75,
120,170,1091121305541,2001822466,1.10,d,Forward charges,135.0,b,1.376,1.5,1.5,91.95,43.05,
121,171,1091121666133,2001823564,0.70,d,Forward and RTO charges,172.8,d,0.672,1.0,1.0,176.3,-3.5,
122,172,1091121981575,2001825261,1.60,d,Forward and RTO charges,345.0,d,1.557,2.0,2.0,352.6,-7.6,


In [256]:

df_invoice_pincode_zone_slab_wt.to_csv('order_level.csv', index = False)

In [257]:

df_invoice_pincode_zone_slab_wt.to_excel('order_level.xlsx', sheet_name = "order_level")

##Summary Dataframe

In [258]:
# created new column named Charge Status and assign three categories based on the difference in charged Amount of Company X and Courier company


for i in range(len(df_invoice_pincode_zone_slab_wt)):
    value=df_invoice_pincode_zone_slab_wt.loc[i, "Difference Between Expected Charges and Billed Charges(Rs.)"]
    # print(value)
    if value>0:
      st="overcharged"
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Charge Status"] = st

    elif value<0:
      st1="undercharged"
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Charge Status"] = st1

    elif value==0:
      st2="correctly charged"
      df_invoice_pincode_zone_slab_wt.loc[df_invoice_pincode_zone_slab_wt.index[i], "Charge Status"] = st2

print(df_invoice_pincode_zone_slab_wt)

    

     index       AWB Code    Order ID  Total weight as per Courier Company  \
0        0  1091117221940  2001806210                                 2.92   
1        1  1091117222065  2001806226                                 0.68   
2        2  1091117222080  2001806229                                 0.71   
3        3  1091117222124  2001806232                                 1.30   
4        4  1091117222135  2001806233                                 0.78   
..     ...            ...         ...                                  ...   
119    169  1091121183730  2001821995                                 0.50   
120    170  1091121305541  2001822466                                 1.10   
121    171  1091121666133  2001823564                                 0.70   
122    172  1091121981575  2001825261                                 1.60   
123    173  1091122418320  2001827036                                 1.60   

    Delivery zone as per courier company         Type of Shipme

In [259]:
# created new df containing two columns: Charge Status and Difference between amounts

df_invoice_pincode_zone_slab_wt[["Charge Status", "Difference Between Expected Charges and Billed Charges(Rs.)"]]

,Charge Status,Difference Between Expected Charges and Billed Charges(Rs.)
0,overcharged,143.85
1,overcharged,45.1
2,overcharged,45.1
3,undercharged,-0.3
4,overcharged,30.65
...,...,...
119,overcharged,14.75
120,overcharged,43.05
121,undercharged,-3.5
122,undercharged,-7.6


In [260]:
# used groupby function to group Charge Status based on Count

summary_count = df_invoice_pincode_zone_slab_wt[["Charge Status", "Order ID"]].groupby(by=["Charge Status"]).count().reset_index()

In [261]:
# print(summary_count)

In [262]:
# used groupby function to group Charge Status based on sum

summary_amt = df_invoice_pincode_zone_slab_wt[["Charge Status", "Difference Between Expected Charges and Billed Charges(Rs.)"]].groupby(by=["Charge Status"]).sum().reset_index()

In [263]:
# print(summary_amt)

In [264]:
# merged two df

dfinal = summary_count.merge(summary_amt, on="Charge Status", how = 'right')

In [265]:
dfinal

,Charge Status,Order ID,Difference Between Expected Charges and Billed Charges(Rs.)
0,correctly charged,9,0.0
1,overcharged,85,4492.0
2,undercharged,30,-644.8


In [266]:
# rename columns

dfinal.rename(columns = {"Difference Between Expected Charges and Billed Charges(Rs.)":'Amount (Rs)'}, inplace = True)
dfinal.rename(columns = {"Order ID":'Count'}, inplace = True)

In [267]:
dfinal

,Charge Status,Count,Amount (Rs)
0,correctly charged,9,0.0
1,overcharged,85,4492.0
2,undercharged,30,-644.8


In [268]:

dfinal.to_csv('summary_table.csv', index = False)

In [269]:

dfinal.to_excel('summary_table.xlsx', sheet_name = "summary_table")